In [1]:
import re
import csv
import numpy as np
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
from DocSim import DocSim

googlenews_model_path = './data/GoogleNews-vectors-negative300.bin'
stopwords_path = "./data/stopwords_en.txt"
cars_path = "./data/train.csv"

In [2]:
df_car = pd.read_csv(cars_path)
df_car['description'] = df_car['description'].str.split('\s?[!,.]\s?')
df_dscpt = df_car[['listing_id', 'description']]

model = KeyedVectors.load_word2vec_format(googlenews_model_path, binary=True)
with open(stopwords_path, 'r') as fh:
    stopwords = fh.read().split(",")
ds = DocSim(model, stopwords=stopwords)

In [10]:
print(df_dscpt.head())

def aggregate(row):
    global items
    _row = row['description']
    if isinstance(_row, list) and len(_row) > 1:
        _tmp = list(filter(str.strip, _row))
        items = [*items, *_tmp]

def counter(row):
    global items_dict, writer
    _dict = {'id':row['listing_id'],
            'years warranty':0,
            'high loan available attractive interest rate':0,
            'well maintained excellent condition accident free non-accident unit':0,
            'fuel efficiency low consumption':0}
    _row = row['description']
    if isinstance(_row, list) and len(_row) > 1:
        _tmp = list(filter(str.strip, _row))
        for doc in _tmp:
            res = ds.calculate_similarity(doc, items)
            if len(res) > 0 and res[0]['score'] > 0.5:
                items_dict[res[0]['doc']] += 1
                if res[0]['doc'] == 'years warranty':
                    num = re.findall(r'(\d+(?:\.\d+)?) years warranty', doc)
                    if len(num) > 0:
                        _dict[res[0]['doc']] = min(num)
                else:
                    _dict[res[0]['doc']] = 1
    writer.writerow(list(_dict.values()))

   listing_id                                        description
0     1030324  [1 owner, 320i gt m-sports model, big brake ki...
1     1021510  [high loan available, low mileage unit, wear a...
2     1026909  [1 owner c&c unit, full agent service with 1 m...
3     1019371  [fully agent maintained, 3 years warranty 10 y...
4     1031014  [kah motor unit, 1 owner, lowest 1, 98% for fu...


In [11]:
items = ['years warranty', 'high loan available attractive interest rate', 'well maintained excellent condition accident free non-accident unit', 'fuel efficiency low consumption']
# items = []
# df_dscpt[:30].apply(aggregate)
# print(len(items))

items_dict = dict.fromkeys(items, 0)


In [12]:
with open('description.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(['listing_id', 'years of warranty', 'better loan offer', 'well maintained', 'low fuel consumption'])
    df_dscpt.apply(counter, axis=1)
items_freq = sorted(items_dict.items(), key=lambda x: x[1], reverse=True)
print(items_freq)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[('high loan available attractive interest rate', 13832), ('well maintained excellent condition accident free non-accident unit', 7039), ('fuel efficiency low consumption', 5340), ('years warranty', 4703)]


In [13]:
freq_dict = dict(items_freq)
with open('description_freq.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(['key phrase', 'count'])
    for key, value in freq_dict.items():
       writer.writerow([key, value])

In [9]:
source_doc = "warranty"
target_docs = ['high loan available attractive interest rate', 'years warranty', 'well maintained excellent condition accident free non-accident unit', 'fuel efficiency low consumption']

sim_scores = ds.calculate_similarity(source_doc, target_docs)

print(sim_scores)

[{'score': 1.0, 'doc': 'years warranty'}, {'score': 0.22199024, 'doc': 'high loan available attractive interest rate'}, {'score': 0.21609513, 'doc': 'well maintained excellent condition accident free non-accident unit'}, {'score': 0.16154926, 'doc': 'fuel efficiency low consumption'}]
